In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import experiment_manager as em
import far_ho as far

Environment variable DATASETS_FOLDER not found. Variables HELP_WIN and HELP_UBUNTU contain info.
Data folder is /Users/Simone/Documents/Pycharm/FAR/tests
Environment variable EXPERIMENTS_FOLDER not found. Current directory will be used
Experiment save directory is  /Users/Simone/Documents/Pycharm/FAR/tests/Experiments
ordered-set package not found..


In [3]:
far.utils._check()

4


In [4]:
try: ss.close()
except: pass
tf.reset_default_graph()
ss = tf.InteractiveSession()
v1 = tf.Variable([1.,3])

v2 = tf.Variable([[-1., -2], [1., 0.]])

In [5]:
lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape())

cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)

io_optim = far.AdamOptimizer(epsilon=1.e-6) # same epsilon of rfho

io_optim_dict = io_optim.minimize(cost) 

oo = tf.reduce_mean(v1*v2)

In [6]:
rhg = far.ReverseHg()
rhg.compute_gradients(oo, io_optim_dict)

optim_oo = tf.train.AdamOptimizer()
ts_hy = optim_oo.apply_gradients(rhg.hgrads_hvars())

In [7]:
tf.global_variables()

[<tf.Variable 'Variable:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'lambda:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'Variable/Adam:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable/Adam_1:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable_1/Adam:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'Variable_1/Adam_1:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'beta1_power_1:0' shape=() dtype=float32_ref>,
 <tf.Variable 'beta2_power_1:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda/Adam:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'lambda/Adam_1:0' shape=(2, 2) dtype=float32_ref>]

In [8]:
far.utils.hyperparameters()

[<tf.Variable 'lambda:0' shape=(2, 2) dtype=float32_ref>]

In [9]:
tf.global_variables_initializer().run()
ss.run(far.utils.hyperparameters())

[array([[ 1.,  1.],
        [ 1.,  1.]], dtype=float32)]

In [10]:
rhg.run(10)

In [11]:
ss.run(far.utils.hypergradients())

[array([[  8.38190317e-07,   6.27245754e-07],
        [ -8.38190317e-07,   0.00000000e+00]], dtype=float32)]

In [12]:
# ss.run(ts_hy)  # perform an hypergradient descent step....
# print(ss.run(far.utils.hyperparameters()))
# print(ss.run(oo))

# check with rfho

In [13]:
import rfho as rf

Environment variable RFHO_EXP_FOLDER not found. Current directory will be used
Experiment save directory is  /Users/Simone/Documents/Pycharm/FAR/tests/Experiments
Environment variable RFHO_DATA_FOLDER not found. Variables HELP_WIN and HELP_UBUNTU contain info.
Data folder is /Users/Simone/Documents/Pycharm/FAR/tests


In [14]:
w, c, co = rf.vectorize_model([v1, v2], cost, oo, augment=2)

In [15]:
dyn = rf.AdamOptimizer.create(w, loss=c)

In [16]:
hyperg = rf.HyperOptimizer(dyn, {co: lmbd}, rf.ReverseHG)

In [17]:
hyperg.initialize()

True

In [18]:
hgs = hyperg.run(10, val_feed_dict_suppliers={co:lambda: None},
                 _debug_no_hyper_update=True)

In [19]:
print('rfho')
print(ss.run(hyperg.hyper_gradients.hyper_gradient_vars))
print('far')
print(ss.run(far.utils.hypergradients()))

rfho
[array([[  1.82008080e-07,   1.36307790e-07],
       [ -1.82029908e-07,  -0.00000000e+00]], dtype=float32)]
far
[array([[  8.38190317e-07,   6.27245754e-07],
       [ -8.38190317e-07,   0.00000000e+00]], dtype=float32)]
